In [6]:
def process_json_list(json_list):
        ldmks = [eval(s) for s in json_list]
        return np.array([(x, 384-y, z) for (x,y,z) in ldmks])

In [7]:
import cv2
import numpy as np
import json
from glob import glob
from tqdm import tqdm
import joblib

json_fns = glob("imgs/*.json")
samples_ns = np.zeros((175000, 2))
target = np.zeros((25000, 2))
i = 0
j = 0
model_x = joblib.load("model_x.pkl")
model_y = joblib.load("model_y.pkl")

In [8]:
for json_fn in tqdm(json_fns[75000:]):
    data_file = open(json_fn)
    data = json.load(data_file)
    #img = cv2.imread("%s.jpg"%json_fn[:-5])
    
    ldmks_interior_margin = process_json_list( data['interior_margin_2d'])
    ldmks_caruncle = process_json_list( data['caruncle_2d'])
    ldmks_iris = process_json_list( data['iris_2d'])
    eye_c = np.mean(ldmks_iris[:,:2], axis=0).astype(int)
    look_vec = list(eval(data['eye_details']['look_vec']))
    look_vec[1] = -look_vec[1]
    norm = np.sqrt(np.sum((np.array([int(ldmks_caruncle[3][0]), int(ldmks_caruncle[3][1])]) - \
                   np.array([int(ldmks_interior_margin[8][0]), int(ldmks_interior_margin[8][1])])) ** 2))
    target[j] = (np.array(eye_c+(np.array(look_vec[:2])*80).astype(int)) - \
                [int(ldmks_caruncle[3][0]), int(ldmks_caruncle[3][1])]) / norm
    sample = (np.array([[int(ldmks_caruncle[3][0]), int(ldmks_caruncle[3][1])],
                      [int(ldmks_interior_margin[2][0]), int(ldmks_interior_margin[2][1])],
                      [int(ldmks_interior_margin[5][0]), int(ldmks_interior_margin[5][1])],
                      [int(ldmks_interior_margin[8][0]), int(ldmks_interior_margin[8][1])],
                      [int(ldmks_interior_margin[11][0]), int(ldmks_interior_margin[11][1])],
                      [int(ldmks_interior_margin[14][0]), int(ldmks_interior_margin[14][1])],
                      list(eye_c)]) - [int(ldmks_caruncle[3][0]), int(ldmks_caruncle[3][1])]) / norm
    samples_ns[i:i+7] = sample
    
    #point_x = model_x.predict(sample.reshape((1, 14)))
    #temp = np.append(sample.reshape((1, 14)), point_x)
    #point_y = model_y.predict(temp.reshape(1, -1))
    #cv2.line(img, tuple(eye_c), tuple([int(point_x * norm + ldmks_caruncle[3][0]),
    #         int(point_y * norm + ldmks_caruncle[3][1])]), (0,255,0), 2)
    
    #cv2.line(img, tuple(eye_c), tuple(eye_c+(np.array(look_vec[:2])*80).astype(int)), (0,255,255), 2)
    #cv2.imshow("syntheseyes_img", img)
    #cv2.waitKey(500)
    #cv2.destroyAllWindows
    i += 7
    j += 1

100%|██████████| 25000/25000 [00:17<00:00, 1466.27it/s]


In [9]:
X = samples_ns.reshape((25000, 14))

In [10]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, target, shuffle=False)

In [12]:
model_x = SVR(C=10, kernel='poly')
model_x.fit(X_train, y_train[:, :1].reshape((y_train.shape[0],)))

SVR(C=10, kernel='poly')

In [13]:
res_x = model_x.predict(X_test)

In [15]:
np.sum(np.abs(res_x - y_test[:, :1].reshape((y_test.shape[0],)))) / y_test.shape[0]

0.04154951211495928

In [16]:
X_for_y = np.hstack((X, target[:, :1]))

In [17]:
X_train_y, X_test_y, y_train_y, y_test_y = train_test_split(X_for_y, target[:, 1:], shuffle=False)

In [18]:
model_y = SVR(C=65, kernel='rbf')
model_y.fit(X_train_y, y_train_y.reshape(-1, 1))

/home/andrey/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR(C=65)

In [20]:
res_y = model_y.predict(X_test_y)
np.sum(np.abs(res_y - y_test_y.reshape((y_test_y.shape[0],)))) / y_test_y.shape[0]

0.031233379531010696

In [23]:
import joblib


In [27]:
joblib.dump(model_x, "model_x.pkl")

['model_x.pkl']

In [28]:
joblib.dump(model_y, "model_y.pkl")

['model_y.pkl']